<a href="https://colab.research.google.com/github/Ken-huea/Language-processing/blob/main/%E6%B8%85%E6%9B%B8%E7%94%A8_janome%E2%91%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 問題　単語のベクトル化
まずこの問題の意味が分からない。[Aidemy教材はこちら](https://premium.aidemy.jp/courses/5050)

それに関する[質問はこちら](https://aidemy-communication.slack.com/archives/C03ANT80SAE/p1662364871248359)

<font color="red">分類問題</font>です。
- ①複数の記事を取得し、必要な部分を抽出。
- ②それを訓練データとテストデータに分割
- ③訓練データを用いてベクトル化する変成器を作成。その際に文章全体ではなく特定の品詞をpic kup してベクトル化を行う。
- ④その変成器を用いて学習 精度確認
- ⑤未知のデータを予測させて、結果を確認。

ポイント
- Tfidfを用いてベクトル化できているか
- ベクトル化したデータで分類モデルを作成できているか
- `tokenize`関数内で特定品詞の抽出が実装できているか

[こちらの教材が近いのか？](https://premium.aidemy.jp/courses/5050/exercises/Hk-uoPwYlG)

##### **この添削問題では、データセットを使用しますので、下記コードを実行ください**

In [ ]:
# 初めの一回だけこのセルと次のセルを実行してください、データセットをダウンロードして展開します
# 一回実行すれば、データセットはダウンロードされたままなので、再起動後等再び実行する必要はありません
import urllib.request
import zipfile

# URLを指定
url = "https://storage.googleapis.com/tutor-contents-dataset/5050_nlp_data.zip"
save_name = url.split('/')[-1]

# ダウンロードする
mem = urllib.request.urlopen(url).read()

# ファイルへ保存
with open(save_name, mode='wb') as f:
    f.write(mem)

# zipファイルをカレントディレクトリに展開する
zfile = zipfile.ZipFile(save_name)
zfile.extractall('.')

In [ ]:
# !pip install Janome==0.3.7
!pip install janome
# ライブラリ読み込み　今回はジャノメ
from janome.tokenizer import Tokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.7 MB 1.5 MB/s 


## 実装

#### ディレクトリから記事のパス取得と読み込み
- ①ディレクトリから指定記事のパスを一括取得
- ②その中からどのくらい実際に読み込むのか選択し読込。
- ③返り値をリストに格納

In [ ]:
import glob
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from janome.tokenizer import Tokenizer

抽出する記事を辞書型にして、keyにディレクトリ名、valueに番号を入力。
　
例えば「dokujo-tsushin」の記事は１番です。みたいに。

そして入れ物にするリストも設定します。今回は可読性優先の為、リストは独自関数の外に出しておりますが、<font color="red">独自関数に組み込む</font>と、独自関数が呼び出されるたびにリスト内容がリセットされます。こちらの方がミスは少なるかもしれません

In [ ]:
# 可読性優先の為独自関数の外に出しておきます。

docs = []          # 記事内容格納リスト
labels = []        # 記事ラベル格納リスト

# ちゃんとkeyの値はディレクトリ名に則て下さい。後でそのkeyを変数名として呼び出しますので。
category = {
        'dokujo-tsushin': 1,             # 'dokujo-tsushin'のラベルは１番
        'it-life-hack': 2,               # 'it-life-hack'のラベルは   2番
        'kaden-channel': 3,              # 'kaden-channel'のラベルは  3番
        'livedoor-homme': 4,             # 'livedoor-homme'のラベルは 4番
        'movie-enter': 5,
        'peachy': 6,
        'smax': 7,
        'sports-watch': 8,
        'topic-news': 9
    }

独自関数です。詳細は[こちらのcolab](https://colab.research.google.com/drive/1OgKT6Wl121ySOWemNRCWqjZsl21Hdicu#scrollTo=C-RgQAK_bciR)を確認してください。

相違点
- 理解やイメージの為に所どころprint文を挿入しております。
- 若干の変数名の変更 `category_key　category_value`

下記の<font color="red">独自関数の概要</font>
- 記事のtxtファイルのパスを取得しております。その記事の内容は上の辞書を参照。それをfilesに格納。
- 実際に読み込むのは３つだけ。その10個を<font color="red">for文とwith open</font>で読み込んでいます。しかもそのまま読み込むのではなく<font color="red">.splitlines()</font>を用いて元記事の改行毎に分割しています。そうすると、使い勝手が良くなります。
実際に記事内容は変数bodyに格納されていますけど[:3]のように１と２を省いています。「１番目２番目は余分だからカットしている」という事です。

- docsのリストには[subjectとbody]　labelsのリストには[labels]が格納されております。


その長くて複雑な独自関数の理解の為に<font color="red">print文</font>の挿入をしているのです。

In [ ]:
def load_livedoor_news_corpus():
  print(category.items())
  for category_key, category_value in category.items():                         # keyとvaleを折れぞれ変数に格納。
    print(category_value)
    files = glob.glob(f"./5050_nlp_data/{category_key}/{category_key}*.txt")    # keyと同一名のディレクトリに格納されているテキストファイル全てのパスを取得。
    text = ''                                                                   # 変数textの初期化(合っても無くてもかまわない)
    for file in files[:10]: # 実行時間の関係上読み込むデータを制限しています。
      print(f'file:{file}')                                                     # file pathを表示。
      with open(file, 'r', errors='ignore') as f:
          lines = f.read().splitlines()
          print("Lines:",lines)                                                 # 読み込んだ内容を全表示。
          # 1,2行目に書いたあるURLと時間は関係ないので取り除きます。
          # url = lines[0]
          # print("url:",url)
          
          # datetime = lines[1]
          # print("datetime:",datetime)
          subject = lines[2]
          print("subject:",subject)                                             # タイトルの主題は[2]番目に格納されてます。それを表示。
          body = "".join(lines[3:])                                             # 本文は[3]以降。
          print("body:",body)
          text = subject + body                                                 # 主題＋本文　(変数で表示)
          print("text:",text)
          print()

      docs.append(text)                                                         # 主題＋本文を格納。
      labels.append(category_value)

  return docs, labels

docs,labels = load_livedoor_news_corpus()


dict_items([('dokujo-tsushin', 1), ('it-life-hack', 2), ('kaden-channel', 3), ('livedoor-homme', 4), ('movie-enter', 5), ('peachy', 6), ('smax', 7), ('sports-watch', 8), ('topic-news', 9)])
1
file:./5050_nlp_data/dokujo-tsushin/dokujo-tsushin-6572058.txt
Lines: ['http://news.livedoor.com/article/detail/6572058/', '2012-05-18T14:00:00+0900', '私、職場での呼び名に悩んでいます', '昨年の秋、転職した坂本エリコさん（28歳・不動産関連）の悩みは職場での呼び名。同僚たちは名前で呼び合っていたり「ちゃん付け」だったりするのだが、エリコさんだけはいつまでたっても「坂本さん」と呼ばれている。', '「同じ時期に転職した同僚は周囲から『メグちゃん』って呼ばれているんです。なのに私だけいつまでたっても『坂本さん』というのは、やっぱり職場に馴染めていないような気がして…。同僚と一緒にお昼を食べているし、飲み会にもよく誘われるので気にしなくてもいいのかもしれませんが…」とエリコさんは話してくれた。', '', '呼び名以外で特に問題が無いのなら、気にする必要もないかもしれない。ただ呼び名は人間関係の距離感が現れるものだけに、エリコさんの悩みも理解できる。', '', 'エリコさんの悩みを聞いて思い出したのが学生時代の友人・土金ムツミさん（仮名/27歳・IT関連）の話だ。', '', 'ムツミさんの職場は上司部下関係なく「さん付け」が基本となっているが、飲み会などではニックネームか「ちゃん付け」になる。 以前は飲み会でも「土金（つちがね）さん」と呼ばれていたムツミさんだが、たまたまメガネをコンタクトレンズに髪型をショートカットにしたところニックネーム呼びに変わったという。', '', '「学生時代に『ドキンちゃん』って呼ばれていたことを話したら、その日からドキンちゃんになりました。前にも飲み会でニックネームの話はしていたんですけれど「土金だからドキ

In [ ]:
# 取得した情報を確認してみましょう

print(len(labels))  # 90
print(labels)

print(len(docs))    # 90
docs

90
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
90


['私、職場での呼び名に悩んでいます昨年の秋、転職した坂本エリコさん（28歳・不動産関連）の悩みは職場での呼び名。同僚たちは名前で呼び合っていたり「ちゃん付け」だったりするのだが、エリコさんだけはいつまでたっても「坂本さん」と呼ばれている。「同じ時期に転職した同僚は周囲から『メグちゃん』って呼ばれているんです。なのに私だけいつまでたっても『坂本さん』というのは、やっぱり職場に馴染めていないような気がして…。同僚と一緒にお昼を食べているし、飲み会にもよく誘われるので気にしなくてもいいのかもしれませんが…」とエリコさんは話してくれた。呼び名以外で特に問題が無いのなら、気にする必要もないかもしれない。ただ呼び名は人間関係の距離感が現れるものだけに、エリコさんの悩みも理解できる。エリコさんの悩みを聞いて思い出したのが学生時代の友人・土金ムツミさん（仮名/27歳・IT関連）の話だ。ムツミさんの職場は上司部下関係なく「さん付け」が基本となっているが、飲み会などではニックネームか「ちゃん付け」になる。 以前は飲み会でも「土金（つちがね）さん」と呼ばれていたムツミさんだが、たまたまメガネをコンタクトレンズに髪型をショートカットにしたところニックネーム呼びに変わったという。「学生時代に『ドキンちゃん』って呼ばれていたことを話したら、その日からドキンちゃんになりました。前にも飲み会でニックネームの話はしていたんですけれど「土金だからドキンなのね」くらいの反応でまったく定着しなかったんですよ（苦笑）。とりあえず上司や同僚との距離も縮まった気がして嬉しかったです」とムツミさんは話してくれた。ところで呼び名から恋愛関係が分かってしまうことは多い。残業を終えたイクミさん（34歳・家電メーカー）が会社近くお店で食事をしていたところ、たまたまサブマネージャーのTさん（36歳）と同僚のM子さん（28歳）が、近くの席に座った。イクミさんに気づかないまま話し始めた二人…。「ウチの職場では残業後に一緒にお茶や食事をすることは珍しくないんですけど、TさんとM子は、み〜ちゃん、たあ君って呼び始めたんです。ビックリしましたよ。いつの間にそういう関係に！ って感じでした。今では職場でも公認のカップルで、オンオフでの二人の呼び名の切り替えも名物化しています（笑）」とイクミさん。身近なところで同様の体験をしている人も

#### データを訓練データと検証データに分割

In [ ]:
# 再度取得した情報を確認してみましょう
print(docs[0])
print("docsにある記事の数:",len(docs),"　docsのtype:",type(docs)) #取得記事 9*10で90 list
print()
print(labels[0])
print("labelsの数:",len(labels),"labelsのtype:",type(labels)) #取得記事 9*10で90 list

私、職場での呼び名に悩んでいます昨年の秋、転職した坂本エリコさん（28歳・不動産関連）の悩みは職場での呼び名。同僚たちは名前で呼び合っていたり「ちゃん付け」だったりするのだが、エリコさんだけはいつまでたっても「坂本さん」と呼ばれている。「同じ時期に転職した同僚は周囲から『メグちゃん』って呼ばれているんです。なのに私だけいつまでたっても『坂本さん』というのは、やっぱり職場に馴染めていないような気がして…。同僚と一緒にお昼を食べているし、飲み会にもよく誘われるので気にしなくてもいいのかもしれませんが…」とエリコさんは話してくれた。呼び名以外で特に問題が無いのなら、気にする必要もないかもしれない。ただ呼び名は人間関係の距離感が現れるものだけに、エリコさんの悩みも理解できる。エリコさんの悩みを聞いて思い出したのが学生時代の友人・土金ムツミさん（仮名/27歳・IT関連）の話だ。ムツミさんの職場は上司部下関係なく「さん付け」が基本となっているが、飲み会などではニックネームか「ちゃん付け」になる。 以前は飲み会でも「土金（つちがね）さん」と呼ばれていたムツミさんだが、たまたまメガネをコンタクトレンズに髪型をショートカットにしたところニックネーム呼びに変わったという。「学生時代に『ドキンちゃん』って呼ばれていたことを話したら、その日からドキンちゃんになりました。前にも飲み会でニックネームの話はしていたんですけれど「土金だからドキンなのね」くらいの反応でまったく定着しなかったんですよ（苦笑）。とりあえず上司や同僚との距離も縮まった気がして嬉しかったです」とムツミさんは話してくれた。ところで呼び名から恋愛関係が分かってしまうことは多い。残業を終えたイクミさん（34歳・家電メーカー）が会社近くお店で食事をしていたところ、たまたまサブマネージャーのTさん（36歳）と同僚のM子さん（28歳）が、近くの席に座った。イクミさんに気づかないまま話し始めた二人…。「ウチの職場では残業後に一緒にお茶や食事をすることは珍しくないんですけど、TさんとM子は、み〜ちゃん、たあ君って呼び始めたんです。ビックリしましたよ。いつの間にそういう関係に！ って感じでした。今では職場でも公認のカップルで、オンオフでの二人の呼び名の切り替えも名物化しています（笑）」とイクミさん。身近なところで同様の体験をしている人も多い

データ分割

<font color="red">train_test_split</font> を行います。　この分割方法を用いればshffleする手間を省略できますので。

しかしながら、上記のように<font color="red">docsとlabelsがlist</font>なので、<font color="red">numpyに変換</font>する必要があります。
そうでないとtrain_test_splitがエラーを吐きますので。

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# リストからnumpyへ。 
docs2 = np.array(docs)
labels2 = np.array(labels)

# データ分割 比率は9:1
x_train , x_test,y_train,y_test = train_test_split(docs2,labels2, test_size=0.1)

# shapeも確認
x_train.shape , x_test.shape,y_train.shape,y_test.shape

((81,), (9,), (81,), (9,))

### 単語をベクトル化しての一連の流れ。
tf-idfを用いて、訓練データとテストデータをベクトル化します。


<font color="red">質問あり</font>
[BOW tf-idfによる重み付け](https://premium.aidemy.jp/courses/5050/exercises/HydX93IoUxz)　　
[こちらのslack](https://aidemy-communication.slack.com/archives/C03ANT80SAE/p1662207583380019?thread_ts=1662203811.211619&cid=C03ANT80SAE)

[こちらのslack](https://aidemy-communication.slack.com/archives/C03ANT80SAE/p1662204154793969?thread_ts=1662202837.471209&cid=C03ANT80SAE)　正規表現としては、(?u)は Unicode、\bは空白、\w は文字、+は直前の文字が1文字以上続く意味で、空白で挟まれた文字列を抽出する意味

In [ ]:
# これで記事のラベル付けと必要部分の抽出,訓練用とテスト用に分割する所まで行けました。　shapeを確認してみましょう。
x_train.shape , x_test.shape,y_train.shape,y_test.shape

((81,), (9,), (81,), (9,))

独自関数
- 記事に対して<font color="red">形態素解析</font>を行います。
- そしてその結果から<font color="red">「名詞」「動詞」「形容詞」「形容動詞」</font>のみ抽出します。

In [ ]:
# 単語の抽出
t=Tokenizer()

def tokenize2(text):
    tokens = t.tokenize(text)
    words = []
    for token in tokens:
        # 「名詞」「動詞」「形容詞」「形容動詞」を取り出してください
        partOfSpeech = token.part_of_speech.split(',')[0]
        if partOfSpeech == '名詞':
            words.append(token.surface)
        if partOfSpeech == '動詞':
            words.append(token.surface)
        if partOfSpeech == '形容詞':
            words.append(token.surface)
        if partOfSpeech == '形容動詞':
            words.append(token.surface)
    return words

In [ ]:
print(x_train[0])      # x_trainの要素の一部を確認

tokenize2(x_train[0])  # 同一のx_trainの要素の一部を上記の独自関数にかけてみたらどうなるのかを確認。
# 形態素解析されて、その中で特定の品詞のみが抽出されておりますね。

【話題】シニア世代のスマートフォン所有率は４.７％各キャリアから様々なスマートフォンが登場している。中には、二つ折りの携帯電話と同じようなタイプのものもあり、多くの人に抵抗なく普及しているといえるだろう。先月、カカクコムが発表したリサーチによると、スマートフォンの所有者は半年前に比べ12.6ポイントも上昇しているということだ。ではシニア世代への普及はどうなのだろう？NTTレゾナントが運営する「goo リサーチ」は「シニアの情報端末保有状況に関する調査」についてアンケートを実施した。これは60歳以上の消費者モニターを対象にしており有効回答者数は8,393名だったという。普段よく利用している情報端末は「テレビ」が一番多く87.0%。続く「固定電話」は73.3%で「通常の携帯電話・PHS」は70.2%だ。やはり従来の情報端末利用率が高いといえる。また、近年贈り物などで増えてきたデジタル情報端末「デジタルフォトフレーム」は8.0%だった。比べて一般で普及が進んでいる「スマートフォン」は4.7%、「タブレット型端末」は2.4%と、所有率は低かった。スマートフォン所有者をさらに詳しく見てみると、アプリをダウンロードした人の割合は無料・有料アプリあわせて86.1%だ。このうち有料アプリの割合は38.9％だった。現在、スマートフォンの所有者は多いとは言えないが、今後さらに使いやすいスマートフォンが登場すると予測されるため、シニア層についても普及が広がる可能性が多いにあるだろう。スマートフォン所有率：41.9%、前回調査（2011年1月）から12.6ポイント上昇。急速に普及進む（カカクコム）パソコン利用者の60歳以上、約8300人対象「シニアの情報端末保有状況」に関する調査結果（NTTレゾナント）    ■関連記事・パナソニック、人気の3Dビエラ「GT3シリーズ」に55V型も登場・シャープ、チューナー分離の薄型テレビ「フリースタイル　アクオス」発表・キーワードはスカパー！この秋パナソニック、シャープ、ソニーからブルーレイレコーダー新機種が続々登場・放射線計測の基礎知識をおさらいしてみよう【ガイガーカウンター特集：豆知識】・東芝からロボット掃除機「Ｓｍａｒｂｏ(スマーボ)　ＶＣ−ＲＢ１００」が発売に


['話題',
 'シニア',
 '世代',
 'スマート',
 'フォン',
 '所有',
 '率',
 '４',
 '.',
 '７',
 '％',
 'キャリア',
 '様々',
 'スマート',
 'フォン',
 '登場',
 'し',
 'いる',
 '中',
 '二',
 'つ折り',
 '携帯',
 '電話',
 'よう',
 'タイプ',
 'もの',
 'あり',
 '多く',
 '人',
 '抵抗',
 'なく',
 '普及',
 'し',
 'いる',
 'いえる',
 '先月',
 'カカクコム',
 '発表',
 'し',
 'リサーチ',
 'よる',
 'スマート',
 'フォン',
 '所有',
 '者',
 '半年',
 '前',
 '比べ',
 '12',
 '.',
 '6',
 'ポイント',
 '上昇',
 'し',
 'いる',
 'こと',
 'シニア',
 '世代',
 '普及',
 'の',
 'NTT',
 'レゾナント',
 '運営',
 'する',
 'goo',
 'リサーチ',
 'シニア',
 '情報',
 '端末',
 '保有',
 '状況',
 '調査',
 'アンケート',
 '実施',
 'し',
 'これ',
 '60',
 '歳',
 '以上',
 '消費',
 '者',
 'モニター',
 '対象',
 'し',
 'おり',
 '有効',
 '回答',
 '者',
 '数',
 '8',
 ',',
 '393',
 '名',
 'いう',
 '普段',
 '利用',
 'し',
 'いる',
 '情報',
 '端末',
 'テレビ',
 '一番',
 '多く',
 '87',
 '.',
 '0',
 '%。',
 '続く',
 '固定',
 '電話',
 '73',
 '.',
 '3',
 '%',
 '通常',
 '携帯',
 '電話',
 'PHS',
 '70',
 '.',
 '2',
 '%',
 '従来',
 '情報',
 '端末',
 '利用',
 '率',
 '高い',
 'いえる',
 '近年',
 '贈り物',
 '増え',
 'き',
 'デジタル',
 '情報',
 '端末',
 'デジタルフォトフレーム',
 '8',
 '.',
 '0',
 '%',
 '

今度はベクトル化(tf-idf)を行っていきます。

- ベクトル表現化を行う変換器を生成します。
- 引数の「<font color="red">tokenizer=tokenize2</font>」は<font color="blue">ベクトル変換を行う際にこの独自関数を通す</font>という事です。

つまり分かち書きされ,「名詞」「動詞」「形容詞」「形容動詞」を抽出された後にベクトル変換を行うという事。

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=tokenize2)

- ベクトルに変換する際に、x_trainを材料として学習させます。<font color="red">fit_transform</font>。これにより｢どの単語はどのようにベクトル化するか｣を学習しております。
- 検証データの場合は,訓練データの fit() の結果に基づくので、 transform()関数 を行う必要があります。　⇒[こちら](https://premium.aidemy.jp/courses/5050/exercises/B1fusDDtef)

In [ ]:
# ベクトルに変換 ⇒ fitは学習する
x_train_matrix = vectorizer.fit_transform(x_train)  # fit及びtransform  fit で学習させています。これにより｢どの単語はどのようにベクトル化するか｣を学習しております。
x_test_matrix = vectorizer.transform(x_test)        # transformのみ。　その上記の学習に則って x_test をベクトル化しています。　そして何よりテストデータはfitしてはダメでしょう。

これによりベクトル変換機、`vectorizer`は完成しました。


### 学習と精度評価

今度は実際に学習を行っていきます。

- x_train_matrix：ベクトル化した記事(名詞/動詞/形容詞/等。)

- y_train：ラベル

In [ ]:
# ナイーブベイズを用いて分類をおこなってください。
clf = MultinomialNB()
clf.fit(x_train_matrix, y_train)

# ランダムフォレストを用いて分類をおこなってください
clf2 = RandomForestClassifier()
clf2.fit(x_train_matrix, y_train)

RandomForestClassifier()

In [ ]:
# 分類結果を表示
print(clf.score(x_train_matrix, y_train))
print(clf.score(x_test_matrix, y_test))

print(clf2.score(x_train_matrix, y_train))
print(clf2.score(x_test_matrix, y_test))

1.0
0.4444444444444444
1.0
0.7777777777777778


これで実際の学習機の評価は出来ました。
　今回の論点は流れを把握する事なので精度は問題ありません。

#### 実際の確認
実際に未知の記事を学習機に取り込ませて、予測させてみましょう。その予測が合っているのでしょうか。

データ読み込み。今回は<font color="red">xmax(label:7)</font>から適当にパスを拾い、読み込みました。読み込んだ内容は<font color="red">読み込み時の独自関数のコード</font>を拝借。

In [ ]:
with open("/content/5050_nlp_data/smax/smax-6606357.txt", 'r', errors='ignore') as f:
          lines = f.read().splitlines()
          print("Lines:",lines)
          # 1,2行目に書いたあるURLと時間は関係ないので取り除きます。
          url = lines[0]
          print("url:",url)
          
          datetime = lines[1]
          print("datetime:",datetime)
          subject = lines[2]
          print("subject:",subject)
          body = "".join(lines[3:])
          print("body:",body)
          text = subject + body
          print("text:",text)
          print()

Lines: ['http://news.livedoor.com/article/detail/6606357/', '2012-05-29T18:35:00+0900', 'ウィルコム初Androidスマートフォン！PHSと3Gにデュアル対応した「DIGNO DUAL WX04K」を写真と動画で紹介【レポート】', 'ウィルコム初のAndroidスマートフォンを探る！ ', '', '既報通り、ウィルコムとソフトバンクモバイルは29日、2012年夏以降に発売する予定の新モデル「ソフトバンクモバイル・ウィルコム新商品発表会 2012 Summer」を開催し、音声通話はPHS＋3Gによる通話と3Gによるデータ通信に対応したデュアルネットワークなAndroidスマートフォン「DIGNO DUAL WX04K」が発表されました。発売は2012年6月21日予定。', '', 'DIGNO DUAL WX04Kは従来のようなPHS端末とは違い、「通話の選択肢にPHSを選んで利用することができる3Gスマートフォン」ともいうべき端末で、データ通信に絡む機能は他キャリアなどで提供されている3G通信のスマートフォンと同様で、音声通話にPHSを利用することでPHS（ウィルコム）の強みであるクリアな音声品質や「誰とでも定額」の恩恵を受ける事が出来るスマートフォンとなっています。', '', '発表会場にて実機の展示がありましたので紹介します。', '', '■デュアルだけど実はトリ！？', '“DIGNO DUAL”というペットネームではありますが、通話にはPHSモード・3Gモードの他にGSMにも対応したモードが用意されており、実はトライネットワーク機であったりします。', '', '', 'DIGNO DUAL WX04K', '', '本体カラーはホワイト・ピンク・グリーン・ブラックの4色', '', 'サイズは、大きさが約130×64×10.7mm（最薄部）、質量が約148g。（バッテリー装着時）OSにはAndroid 2.3（開発コード名：Gingerbread)を採用していますが、発売後にAndroid 4.0（開発コード名：IceCream Sandwich；ICS）へのアップデート予定がとのことです。', '', 'CPUは1.2GHzデュアルコアのルネサス製「MP522

In [ ]:
# 形式を合わせる必要がありそうなのでtypeを色々確認してみましょう。

# 上記の内容からtextを用いれば良さそうですね。
print(type(text))         # こっちはstr
print(type(x_train[0]))   # こっちは'numpy.str_'
print(type(x_test[0]))   # こっちは'numpy.str_'

text2 = np.array(text)
print(type(text2))         # こっちはstr
text2

<class 'str'>
<class 'numpy.str_'>
<class 'numpy.str_'>
<class 'numpy.ndarray'>


array('ウィルコム初Androidスマートフォン！PHSと3Gにデュアル対応した「DIGNO DUAL WX04K」を写真と動画で紹介【レポート】ウィルコム初のAndroidスマートフォンを探る！ 既報通り、ウィルコムとソフトバンクモバイルは29日、2012年夏以降に発売する予定の新モデル「ソフトバンクモバイル・ウィルコム新商品発表会 2012 Summer」を開催し、音声通話はPHS＋3Gによる通話と3Gによるデータ通信に対応したデュアルネットワークなAndroidスマートフォン「DIGNO DUAL WX04K」が発表されました。発売は2012年6月21日予定。DIGNO DUAL WX04Kは従来のようなPHS端末とは違い、「通話の選択肢にPHSを選んで利用することができる3Gスマートフォン」ともいうべき端末で、データ通信に絡む機能は他キャリアなどで提供されている3G通信のスマートフォンと同様で、音声通話にPHSを利用することでPHS（ウィルコム）の強みであるクリアな音声品質や「誰とでも定額」の恩恵を受ける事が出来るスマートフォンとなっています。発表会場にて実機の展示がありましたので紹介します。■デュアルだけど実はトリ！？“DIGNO DUAL”というペットネームではありますが、通話にはPHSモード・3Gモードの他にGSMにも対応したモードが用意されており、実はトライネットワーク機であったりします。DIGNO DUAL WX04K本体カラーはホワイト・ピンク・グリーン・ブラックの4色サイズは、大きさが約130×64×10.7mm（最薄部）、質量が約148g。（バッテリー装着時）OSにはAndroid 2.3（開発コード名：Gingerbread)を採用していますが、発売後にAndroid 4.0（開発コード名：IceCream Sandwich；ICS）へのアップデート予定がとのことです。CPUは1.2GHzデュアルコアのルネサス製「MP5225」を採用。本体メモリは2GB、RAMは1GBとなっています。この他にもIPX5、IPX7相当の防水機能を備えています。表面と裏面表側は機能・ホーム・バック・検索の4キー配置となっている他、スピーカーや近接・照度センサーに約30万画素CMOSのインカメラを備えています。裏面には約500万画素CMOSのアウトカメラに

それでは未知の記事データをvectrizerにてベクトル化させてみましょう。　
[先生の回答](https://aidemy-communication.slack.com/archives/C03ANT80SAE/p1662547691082579?thread_ts=1662376146.173589&cid=C03ANT80SAE)　リスト化させると上手く行きました。

In [ ]:
# このtextをまずベクトル化
# text_matrix = vectorizer.transform(text)  # これだとエラーになる。
text_matrix = vectorizer.transform([text])  # リスト化？numpy?　これだと上手く行った？
text_matrix

<1x7076 sparse matrix of type '<class 'numpy.float64'>'
	with 197 stored elements in Compressed Sparse Row format>

In [ ]:
b = [text]
print(type(b))

そして予測をさせてみましょう。

In [ ]:
predict = clf2.predict(text_matrix)
predict # array([7])

# ちゃんと予測ができて、尚且つ正解です。　めでたし。

array([7])

最後に単語単発からでも、ラベルの予測が出来るはずです。　同じ要領で試してみましょう。

In [ ]:
vocab = "太陽"

# vocabをベクトル化
vocab_matrix = vectorizer.transform([vocab]) 

<1x7076 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [ ]:
# 予測。
vocab_matrix_predict = clf2.predict(vocab_matrix)
vocab_matrix_predict

# つまり｢太陽｣という単語は分類分けすると、'it-life-hack'という記事群に格納されていそう。という事ですね。

array([2])